# Tarefa 02 Módulo 05

O nosso projeto desta sequência de módulos do curso será um aprofundamento da demonstração sobre classificação de risco de crédito que vimos lá no comecinho. Pois recebemos uma base já montada pra nós. Tenha certeza de que ela passou por um longo processamento até ficar daquele jeito. Neste exercício vamos exercitar o que aprendemos nas ultimas aulas e montar a variável resposta da base do nosso projeto.

#### Marcação de bom e mau
O objetivo da modelagem é classificar o risco de inadimplência, ou como se diz no meio, o risco de *default*. Podemos fazer longas discussões sobre o conceito de *default* com base em estudos e exigências regulatórias, para efeitos deste estudo, um cliente em *default* é aquele que está em 60 dias de atraso ou mais. Então classificaremos os clientes como 'bons' e 'maus' assim:
- **Maus** pagadores: são aqueles que entraram em 'default' (atraso 60 dias ou mais) nos 24 meses seguintes à aquisição do cartão de crédito.
- **Bons** pagadores: são considerados todos os demais.
- **Excluídos**: Clientes que não adquiriram um cartão de crédito (seja por recusa, seja por desistência) não possuem informações de pagamento, portanto não se pode identificar se são bons ou maus. Há uma longa discussão e literatura sobre *inferência de rejeitados* que está fora do escopo deste exercício.

#### Bases disponíveis
Temos duas bases importantes aqui: uma de propostas, com diversas informações dos vários solicitantes de cartão de crédito, e uma base de pagamentos. A base de pagamentos será utilizada para identificar a ocorrência de *default*. A base de propostas tem diversas informações coletadas no momento da solicitação do crédito (isto é importante: qualquer informação posterior a essa data é impossível de ser coletada na aplicação do modelo e não pode ser utilizada).

As variáveis delas são:

Base de propostas - application_records.csv

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| CODE_GENDER| M = 'Masculino'; F = 'Feminino' |M/F|
| FLAG_OWN_CAR| Y = 'possui'; N = 'não possui' |Y/N|
| FLAG_OWN_REALTY| Y = 'possui'; N = 'não possui' |Y/N|
| CNT_CHILDREN| Quantidade de filhos |inteiro|
| AMT_INCOME_TOTAL| Annual income |inteiro|
| NAME_INCOME_TYPE|Tipo de renda (ex: assaliariado, autônomo etc) | texto |
| NAME_EDUCATION_TYPE| Nível de educação (ex: secundário, superior etc) |texto|
| NAME_FAMILY_STATUS | Estado civil (ex: solteiro, casado etc)| texto |
| NAME_HOUSING_TYPE | tipo de residência (ex: casa/apartamento, com os pais etc) | texto |
| DAYS_BIRTH | Count backwards from current day (0), -1 means yesterday |inteiro|
| DAYS_EMPLOYED | Count backwards from current day (0), -1 means yesterday |inteiro|
| FLAG_MOBIL | Indica se possui celular (1 = sim, 0 = não) |binária|
| FLAG_WORK_PHONE | Indica se possui telefone comercial (1 = sim, 0 = não) |binária|
| FLAG_PHONE | Indica se possui telefone (1 = sim, 0 = não) |binária|
| FLAG_EMAIL | Indica se possui e-mail (1 = sim, 0 = não) |binária|
| OCCUPATION_TYPE | Occupation	 |Qualitativa|
| CNT_FAM_MEMBERS | quantidade de pessoas na residência |inteiro|

Base de pagamentos - pagamentos_largo.csv  

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| mes_00 a mes_24| faixa de atraso mês a mês do cliente <br>0: 1-29 days past due &nbsp;&nbsp;&nbsp;&nbsp; 1: 30-59 days past due <br />2: 60-89 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 3: 90-119 days overdue <br /> 4: 120-149 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 5: more than 150 days <br />C: paid off that month &nbsp;&nbsp;&nbsp;&nbsp; X: No loan for the month |Qualitativa|

#### Construindo a variável resposta
A base de pagamentos está em um formato de 'base larga'. Essa base possui informações de pagamentos do cliente mês a mês a partir do mês de aquisição do crédito (mês 0) até o vigésimo quarto mês após a aquisição do crédito (mês 24). Utilizaremos essa base para determinar se um proponente é considerado 'bom pagador' ou caso apresente atraso representativo, será considerado 'mau pagador'.

#### Base larga vs base longa
A base ser larga significa que há uma linha para cada cliente, e que as informações estarão nas colunas, em contraste com a 'base longa', em que haveria uma linha para cada combinação cliente/mês, uma coluna indicando o cliente, outra indicando o mês, e apenas uma coluna com a informação do atraso.

#### Tarefa 1) Marcar *default* no mês
Faça uma indicadora de se o cliente está em *default* em cada uma das marcações (mes_00 a mes_24). Dica: você pode utilizar o método ```.isin()``` do Pandas. Consulte a [documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) caso necessário.

#### Tarefa 2) 'bons' e 'maus' ao longo de todos os 24 meses de desempenho
Marque para cada cliente se ele teve pelo menos um episódio de *default* entre o mês 0 e o mês 24. Dica: o método ```sum()``` pode ajudar. Caso precise, consulte a [documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) e procure pelo argumento ```axis```, você viu outros métodos que possuem esse argumento também. Tendo o número de meses em default de cada cliente, basta marcar ```True``` para todos aqueles que possuem pelo menos 1 mês em *default* e ```False``` para os demais.

#### Tarefa 3) Marcando proponentes expostos ao risco de crédito
Marcando proponentes que se tornaram tomadores: lembre-se de que clientes que não adquiriram o cartão devem ser desconsiderados. A base de pagamentos possui apenas clientes que adquiriram cartão de crédito, então você pode selecionar somente os clientes da base de propostas que se encontram na base de pagamentos.

#### Tarefa 4) Consolidando as informações
Faça uma junção das informações da base de propostas com a variável de *default* que você acabou de construir. Talvez você consiga realizar a tarefa 3 e tarefa 4 em uma única linha de código ;)

#### Tarefa 5) Verificando
Faça uma contagem dos valores do *default* que você construiu.

In [1]:
import pandas as pd
import numpy as np


In [2]:
propostas = pd.read_csv('application_record.csv')
pg = pd.read_csv('pagamentos_largo.csv')

In [3]:
pg.head()

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
0,5001718,0,0,0,0,0,0,0,0,NaN,...,NaN,0,NaN,0,0,0,0,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C


In [4]:
# Função para extrair o número da coluna
def extrair_numero(coluna):
    if coluna.startswith('mes_'):
        try:
            return int(coluna.split('_')[1])
        except ValueError:
            return float('inf')  # Caso não seja um número válido
    return float('inf')  # Para não-mes_ colunas

# Obter a lista de colunas
colunas = pg.columns.tolist()

# Separar a coluna 'ID' das outras
id_coluna = 'ID'
colunas_mes = [col for col in colunas if col.startswith('mes_')]

# Ordenar as colunas 'mes_' com base no número
colunas_mes_sorted = sorted(colunas_mes, key=extrair_numero)

# Reconstruir a lista de colunas com 'ID' no início
nova_ordem = [id_coluna] + colunas_mes_sorted

# Reordenar as colunas do DataFrame
pg = pg[nova_ordem]

pg.head()

,ID,mes_0,mes_1,mes_2,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,...,mes_15,mes_16,mes_17,mes_18,mes_19,mes_20,mes_21,mes_22,mes_23,mes_24
0,5001718,0,0,0,0,0,0,0,NaN,0,...,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3,5001723,0,0,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5001726,0,0,0,0,0,0,C,C,C,...,C,C,C,C,C,C,C,C,C,C


In [5]:
#analisando uma amostra do banco de dados

pg.sample(10)

,ID,mes_0,mes_1,mes_2,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,...,mes_15,mes_16,mes_17,mes_18,mes_19,mes_20,mes_21,mes_22,mes_23,mes_24
5698,5028986,0,0,0,0,0,0,0,0,0,...,C,C,C,C,C,C,C,C,C,C
6756,5038782,0,NaN,0,0,0,0,0,0,NaN,...,0,NaN,0,C,C,C,C,C,C,C
19823,5143177,C,C,C,C,C,C,C,C,C,...,C,C,C,C,NaN,NaN,NaN,NaN,NaN,NaN
18958,5135982,0,0,0,0,0,0,0,0,0,...,0,0,0,0,C,C,C,C,C,C
11717,5074583,0,0,C,C,C,C,C,C,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12342,5087989,0,0,0,0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18830,5135607,0,0,0,0,0,0,0,0,0,...,C,C,C,C,C,C,C,C,C,C
2462,5011487,0,NaN,0,NaN,0,0,NaN,NaN,0,...,0,NaN,C,C,C,C,C,C,C,C
9440,5059918,0,0,0,0,0,0,C,C,C,...,C,C,C,C,C,C,C,C,C,C
842,5004105,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 1) Seu código aqui

In [6]:
# Lista de valores que indicam não default
valores_default = ['2','3','4','5']


In [7]:
# Listar todas as colunas de meses
colunas_mes = [col for col in pg.columns if col.startswith('mes_')]

In [8]:
# Criar colunas indicadoras de default usando o método .isin()
for mes in colunas_mes:
    pg[f'{mes}_default'] = pg[mes].isin(valores_default)

pg.head()

,ID,mes_0,mes_1,mes_2,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,...,mes_15_default,mes_16_default,mes_17_default,mes_18_default,mes_19_default,mes_20_default,mes_21_default,mes_22_default,mes_23_default,mes_24_default
0,5001718,0,0,0,0,0,0,0,NaN,0,...,False,False,False,False,False,False,False,False,False,False
1,5001719,0,0,C,C,C,C,C,C,C,...,False,False,False,False,False,False,False,False,False,False
2,5001720,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
3,5001723,0,0,0,0,0,0,0,0,NaN,...,False,False,False,False,False,False,False,False,False,False
4,5001726,0,0,0,0,0,0,C,C,C,...,False,False,False,False,False,False,False,False,False,False


In [9]:
#Validando se há algum valor default
existe_true = np.any(pg == True)
print(existe_true)

True


In [10]:
# 2) Seu código aqui
#lista de colunas default
colunas_default = [col for col in pg.columns if col.endswith('_default')]

In [11]:
#Verificar valores default
pg['default'] = pg[colunas_default].sum(axis=1) > 0

In [12]:
# Converter a coluna para booleano True/False
pg['default'] = pg['default'].astype(bool)

pg.head()

,ID,mes_0,mes_1,mes_2,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,...,mes_16_default,mes_17_default,mes_18_default,mes_19_default,mes_20_default,mes_21_default,mes_22_default,mes_23_default,mes_24_default,default
0,5001718,0,0,0,0,0,0,0,NaN,0,...,False,False,False,False,False,False,False,False,False,False
1,5001719,0,0,C,C,C,C,C,C,C,...,False,False,False,False,False,False,False,False,False,False
2,5001720,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
3,5001723,0,0,0,0,0,0,0,0,NaN,...,False,False,False,False,False,False,False,False,False,False
4,5001726,0,0,0,0,0,0,C,C,C,...,False,False,False,False,False,False,False,False,False,False


,count
default,
False,20506
True,431


In [13]:
propostas.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134.0,1.0,0.0,0.0,0.0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0


In [14]:
# 3) Seu código aqui
#extraindo lista de clientes da base de proposta
lista_ids = propostas['ID'].tolist()

In [15]:
# Marcar os proponentes que estão na lista de clientes com cartão
pg['se_tornou_tomador'] = propostas['ID'].isin(lista_ids)

pg[pg['se_tornou_tomador']==True]

,ID,mes_0,mes_1,mes_2,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,...,mes_17_default,mes_18_default,mes_19_default,mes_20_default,mes_21_default,mes_22_default,mes_23_default,mes_24_default,default,se_tornou_tomador
0,5001718,0,0,0,0,0,0,0,NaN,0,...,False,False,False,False,False,False,False,False,False,True
1,5001719,0,0,C,C,C,C,C,C,C,...,False,False,False,False,False,False,False,False,False,True
2,5001720,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,True
3,5001723,0,0,0,0,0,0,0,0,NaN,...,False,False,False,False,False,False,False,False,False,True
4,5001726,0,0,0,0,0,0,C,C,C,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20932,5150475,C,C,C,C,C,C,C,C,C,...,False,False,False,False,False,False,False,False,False,True
20933,5150476,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,True
20934,5150480,0,0,0,0,0,0,C,C,C,...,False,False,False,False,False,False,False,False,False,True
20935,5150482,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,True


In [18]:
#unindo os banco de dados

propostas['default'] = propostas.merge(pg,on='ID',how='left')['default'].fillna(0)



In [19]:
# 5) eu código aqui
pg['default'].value_counts()

,count
default,
False,20506
True,431


In [20]:
# 5) eu código aqui
propostas['default'].value_counts()

,count
default,
0,84510
True,211
